In [1]:
import google.generativeai as genai
import os

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

/home/doosti@chapman.edu/.conda/envs/gemeni/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()

In [3]:
data = pd.read_csv('~/projects/Facebook/top2vec/data/sponsored_videos_data.csv')
data.head()

,video_id,new_id,creator_id,creator_name,sponsor_id,sponsor_name,title_description,topics,labels,labels2,transcript,text
0,3630692966420,3630692966420,EqB7bWPKAq,Axios,HLchLkbYxo,GE,Thanks to 9 sq miles of solar panels and the o...,NaN,NaN,NaN,NaN,thanks to 9 sq miles of solar panels and the o...
1,4113787925913,4113787925913,OhoPnkQZgz,Vanity Fair,UwAcLwRAua,Louis Vuitton Official,Explore the world of Masters Take a look at on...,Louis Vuitton;Jeff Koons;Vanity Fair,NaN,NaN,NaN,explore the world of masters take a look at on...
2,126993937922265,126993937922265,2oENn5je9B,Major League Soccer (MLS),1FhTxAR4dc,Continental Tire,ExtraTime Live: Transfer Window winners Who we...,NaN,"['presentation', 'text']","['font', 'graphics', 'logo', 'news conference'...",NaN,extratime live: transfer window winners who we...
3,161554750989183,161554750989183,4G5GeWxsA8,Animal Planet,7fDtbKPniI,Animal Nation with Anthony Anderson,Find out why Anthony Anderson wanted to host a...,NaN,NaN,NaN,NaN,find out why anthony anderson wanted to host a...
4,162004174373759,162004174373759,W6ddNwouXN,MotoGP eSport,ya7I3ARgDC,Casa Tarradellas - PÃÂ¡gina Oficial,Challenge #1 fastest lap Watch the fastest lap...,Grand Prix motorcycle racing;Fastest lap;Video...,NaN,NaN,NaN,challenge #1 fastest lap watch the fastest lap...


In [4]:
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
from google.api_core import retry

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(text: str) -> list[float]:
        # Set the task_type to CLUSTERING.
        embedding = genai.embed_content(model=model,
                                        content=text,
                                        task_type="SEMANTIC_SIMILARITY")['embedding']
        return np.array(embedding)

    return embed_fn

def create_embeddings(df):
    model = 'models/text-embedding-004'
    return df['text'].progress_apply(make_embed_text_fn(model)).tolist()

In [33]:
# create text column
import ast
vca = data[data.labels.notnull()].copy()
text = []
for i, row in vca.iterrows():
    labels = ast.literal_eval(row['labels'])
    labels2 = ast.literal_eval(row['labels2'])
    if type(row['topics'])!=str:
        topics = []
    else:
        topics = row['topics'].lower().split(';')
    text_set = set(labels + labels2 + topics)
    text.append(" ".join(list(text_set)))
vca['text'] = text
#vca['text'] = vca['labels'].fillna('').apply(lambda x: " ".join(ast.literal_eval(x))) + " " + vca['labels2'].fillna('').apply(lambda x: " ".join(ast.literal_eval(x)))
vca[['labels','labels2','topics','text']].sample(5)

,labels,labels2,topics,text
23845,"['ball game', 'baseball field', 'sport venue',...","['player', 'sign', 'trademark', 'signage', 'ba...",NaN,team sport emblem banner ball game graphics si...
19578,"['graphics', 'brand', 'graphic design', 'text'...","['font', 'logo', 'graphics', 'brand', 'graphic...",Bleacher Report;Terrelle Pryor,font graphics logo graphic design terrelle pry...
9367,"['ball game', 'protective gear in sports', 'so...","['team sport', 'sports', 'football helmet', 'f...",NaN,soccer team sport football helmet ball game am...
2155,['physician'],"['ankle', 'cream', 'solution', 'song', 'physic...",NaN,physician hat cream cosmetics hair thumb water...
32685,"['dish', 'recipe', 'food', 'cooking', 'song']","['dish', 'snack', 'dessert', 'song', 'animatio...",NaN,dessert baking advertising ice cream animation...


In [34]:
print(vca.shape)
print(vca.notnull().sum())

(9519, 12)
video_id             9519
new_id               9519
creator_id           9519
creator_name         9519
sponsor_id           9515
sponsor_name         9519
title_description    9519
topics               3754
labels               9519
labels2              9519
transcript           4484
text                 9519
dtype: int64


In [35]:
batch_size = 500
embeddings = []
for i, batch in enumerate(np.array_split(vca, len(vca) // batch_size)):
    print(f'Processing batch {i + 1}...')
    embeddings.extend(create_embeddings(batch))

# df = create_embeddings(data)
# df.drop('index', axis=1, inplace=True)

/home/doosti@chapman.edu/.conda/envs/gemeni/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing batch 1...


  0%|          | 0/501 [00:00<?, ?it/s]

100%|██████████| 501/501 [01:26<00:00,  5.80it/s]


Processing batch 2...


...

100%|██████████| 501/501 [01:25<00:00,  5.86it/s]


Processing batch 18...


100%|██████████| 501/501 [01:25<00:00,  5.85it/s]


Processing batch 19...


100%|██████████| 501/501 [01:25<00:00,  5.86it/s]


In [36]:
# save embeddings
import pickle
with open('/home/doosti@chapman.edu/projects/Facebook/top2vec/data/embeddings_vca.pkl', 'wb') as f:
    pickle.dump(embeddings, f)